In [153]:
with open('H:/College WOrk/papers/stop-words.txt',encoding='utf-16') as fp:
    v = fp.read()

def removestopwords(line):
    #print("original line",line)
    querywords = line.split()

    resultwords  = [word for word in querywords if word.lower() not in v]
    result = ' '.join(resultwords)
    #print(result)
    return result

In [154]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [ ]:
from statistics import mean
def Average(lst):
    return mean(lst)

In [ ]:
from rouge.rouge import rouge_n_sentence_level
from rouge import rouge_n_summary_level
from rouge import rouge_l_summary_level
from rouge import rouge_w_sentence_level
from rouge import rouge_w_summary_level
import os
from urllib.parse import urlparse
def comparerouge1(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a2_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    _, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
    #rouge1.append(rouge_1)
    #print('ROUGE-1: %f' % rouge_1)
    _, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
    #print('ROUGE-2: %f' % rouge_2)
    #rouge2.append(rouge_2)
    _, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-L: %f' % rouge_l)
    #rougel.append(rouge_l)
    _, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
    #avg=(rouge_1+rouge_2+rouge_l+rouge_w)/4
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_1

In [ ]:
def comparerouge2(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a2_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
   
    _, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
   
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_2   

In [ ]:
def comparerougel(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a2_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    
    _, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
    
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_l    

In [ ]:
def comparerougew(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a2_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    _, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
    
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_w

In [155]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize
import pandas as pd
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
import numpy as np

vectorizer = CountVectorizer()

finallist=[]
rouge1=[]
rouge2=[]
rougel=[]
rougew=[]
class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file(filename,createfile):
    tokenize_words=[]
    maintext=[]
    listofimpcandidate=[]
    listwithfinalsentence=[]
    file1 = open(createfile, "w", encoding="utf-8")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    soup = BeautifulSoup(data)
    docno=soup.find_all('docno')
    titleno = soup.find_all('title')
    text=soup.find_all('text')
    instances = {}
    my_id = 0
    for n,tit,maintitle in zip(docno,text,titleno):
        tit=tit.get_text()
        tit=str(tit)
        sentences=sentence_tokenize.sentence_split(tit, lang='gu')
        for t in sentences:
            sen=removestopwords(t)
            #print("this is sen",sen)
            maintext.append(sen)
        list3 = [''.join(c for c in s if c not in string.punctuation) for s in maintext]
        list3 = [s for s in list3 if s]
        
        bag_of_words = vectorizer.fit_transform(list3)
        bag_of_words.todense()
        #This process encodes our original data into topic encoded data
        svd = TruncatedSVD(n_components = 2)
        lsa = svd.fit_transform(bag_of_words)
        topic_encoded_df = pd.DataFrame(lsa, columns=["topic1", "topic2"])
        topic_encoded_df["doc"]= list3
        avg1=topic_encoded_df["topic1"].mean()
        avg2=topic_encoded_df["topic2"].mean()
        topic_encoded_df['topic1'] = np.where(topic_encoded_df['topic1'] < avg1, 0, topic_encoded_df['topic1'])
        topic_encoded_df['topic2'] = np.where(topic_encoded_df['topic2'] < avg2, 0, topic_encoded_df['topic2'])
        sum_column = topic_encoded_df["topic1"] + topic_encoded_df["topic2"]
        topic_encoded_df["sum_column"]=sum_column
        sort_by_sumcolumn = topic_encoded_df.sort_values('sum_column',ascending=False)
        compressionratio = round((len(sort_by_sumcolumn.index)*50)/100)
        #print(compressionratio)
        final_df2=sort_by_sumcolumn['doc'].head(compressionratio)
        listtoretrieve=list(final_df2.index.values)
        listtoretrieve.sort()
        originalsen=[]
        for t in sentences:    
            originalsen.append(t)
        df4 = pd.DataFrame(originalsen,columns=['sentences'])
        #print(df4)
        finaldraft=""
        for i in listtoretrieve:
            finaldraft+=df4['sentences'].loc[i]
        rouge_1=comparerouge1("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/economics/annotator2/",finaldraft,filename)
        rouge_2=comparerouge2("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/economics/annotator2/",finaldraft,filename)
        rouge_l=comparerougel("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/economics/annotator2/",finaldraft,filename)
        rouge_w=comparerougew("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/economics/annotator2/",finaldraft,filename)
        
        rouge1.append(rouge_1)
        rouge2.append(rouge_2)
        rougel.append(rouge_l)
        rougew.append(rouge_w)
        
        rouge1average=Average(rouge1)
        rouge2average=Average(rouge2)
        rougelaverage=Average(rougel)
        rougewaverage=Average(rougew)
        print("rouge1 average",rouge1average)
        print("rouge2 average",rouge2average)
        print("rougel average",rougelaverage)
        print("rougew average",rougewaverage)
        #print(finaldraft)            
#         originalfull_str = convert_list_to_string(finaldraft)
#         originalfull_str=" ".join(originalfull_str.split())
#         n=str(n)
        file1.write(finaldraft)
        instances = {}    
        
    return instances

In [156]:
import os

for root, dirs, files in os.walk("G:/Movies/gujarati Text summarization dataset/entertainment/"):    
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                load_file(os.path.join(root, file),"G:/Movies/gujarati Text summarization_lda/entertainment/"+file)